In [45]:
# Libraries for Web Scraping
from bs4 import BeautifulSoup
import requests

# Libraries for Data Processing
import pandas as pd 
from unidecode import unidecode
from datetime import datetime
import time
import re

# Récupération des données des sets pokémons (Nom du set + cartes associées)

In [46]:
pokemon_set_name_list =[
    'PAF',
    'PAR',
    'MEW',
    'OBF',
    'PAL',
    'SVI',
    'CRZ',
    'SIT',
    'LOR',
    'PGO',
    'ASR',
    'BRS',
    'FST',
    'CEL',
    'EVS',
    'CRS',
    'SWSH5',
    'SWSH45',
    'SWSH4',
    'SWSH35',
    'SWSH3',
    'SWSH2',
    'SWSH1'
    ]

In [47]:
def get_set_name_data(pokemon_set_name_list):
    soup_list = []

    for set_name in pokemon_set_name_list:
        url = f'https://www.pokecardex.com/series/{set_name}'
        
        # Récupérer le contenu de la page
        r = requests.get(url)
        if r.status_code == 200:
            soup = BeautifulSoup(r.text, 'html.parser')
            soup_list.append(soup)
        else:
            print(f"La requête pour {set_name} a échoué avec le code d'état {r.status_code}")

    return soup_list

In [48]:
def parse(soup_list):
    set_data_list = []


    for soup in soup_list:
        
        # Trouver le titre de la série s'il est présent
        set_title_tag = soup.find('h5', class_='d-block text-center mb-3')
        if set_title_tag:
            set_title = set_title_tag.text.strip()

            release_date_tag = soup.find('div', class_='d-flex align-items-center mt-2 d-none d-lg-block justify-content-lg-start')
            if release_date_tag:
                release_date = release_date_tag.text.strip()

            cards_count_tag = soup.find('div', class_='d-flex align-items-center mt-2 justify-content-center justify-content-lg-start mb-3 mb-lg-0') 
            if cards_count_tag:
                total_cards_count  = cards_count_tag.text.strip()
                total_cards_count = total_cards_count.split()[0]

            # Trouver toutes les divisions de carte sur la page
            card_divs = soup.find_all('div', class_='serie-details-carte')

            # Pour chaque division de carte, extraire le titre de la carte et l'associer à la série
            for card_div in card_divs:
                card_title = card_div.find('a').get('title')
                set_data_list.append({'set_name' : set_title, 'card_title' : card_title,'release_date' : release_date, 'cards_count' : total_cards_count })
        else:
            print("Balise de titre de série non trouvée sur la page.")

    return  set_data_list

In [49]:
def output(set_data_list):
    cards_df = pd.DataFrame(set_data_list)
    cards_df.drop_duplicates(inplace=True)
    return cards_df

In [50]:
def set_name_scrap(pokemon_set_name_list):
    soup = get_set_name_data(pokemon_set_name_list)
    soup_list = parse(soup)
    cards_df = output(soup_list)
    cards_df['card_title'] = cards_df['card_title'].apply(unidecode)
    cards_df['card_title'] = cards_df['card_title'].str.lower()
    cards_df['card_title'] = cards_df['card_title'].str.replace('/', ' ')
    cards_df['card_title'] = cards_df['card_title'].str.replace('-', ' ')
    return cards_df

In [52]:
pokemon_set_df = set_name_scrap(pokemon_set_name_list)

Balise de titre de série non trouvée sur la page.


In [53]:
pokemon_set_df

,set_name,card_title,release_date,cards_count
0,Destinées de Paldea,pomdepik 1 91,26/01/2024,245
1,Destinées de Paldea,foretress ex 2 91,26/01/2024,245
2,Destinées de Paldea,maracachi 3 91,26/01/2024,245
3,Destinées de Paldea,terracool 4 91,26/01/2024,245
4,Destinées de Paldea,terracruel ex 5 91,26/01/2024,245
...,...,...,...,...
4610,Épée et Bouclier,zamazenta v 212 202,07/02/2020,216
4611,Épée et Bouclier,ballon 213 202,07/02/2020,216
4612,Épée et Bouclier,ecusson metal 214 202,07/02/2020,216
4613,Épée et Bouclier,canne ordinaire 215 202,07/02/2020,216


# Récupération du nom des Blocs ( Ecarlate et Violet, Epée et Bouclier, etc...)

In [ ]:
def get_bloc_name_data():
    soup_list = []

    
    url = f'https://www.pokecardex.com/series/'
        
    # Récupérer le contenu de la page
    r = requests.get(url)
    if r.status_code == 200:
        soup = BeautifulSoup(r.text, 'html.parser')
        soup_list.append(soup)
    else:
        print(f"La requête pour {set_name} a échoué avec le code d'état {r.status_code}")

    return soup_list

In [ ]:
def bloc_name_parse(soup_list):
    bloc_names_list = []
    for soup in soup_list:
        
        bloc_name_tags = soup.find_all('p', class_='p-0 m-0')
        for bloc_tag in bloc_name_tags:
            bloc_name = bloc_tag.text.strip()
            bloc_names_list.append(bloc_name)
    return bloc_names_list

In [ ]:
def bloc_name_list():
    bloc_soup_list = get_bloc_name_data()
    bloc_name_list = bloc_name_parse(bloc_soup_list)
    
    '''# Liste pour stocker les noms de blocs modifiés
    modified_bloc_name_list = []
    
    for bloc_name in bloc_name_list:
        # Convertir en minuscules et enlever les accents
        modified_bloc_name = unidecode(bloc_name.lower())
        modified_bloc_name_list.append(modified_bloc_name)
    
    modified_bloc_name_list = modified_bloc_name_list[:2]'''
        
    return bloc_name_list[:2]

In [ ]:
bloc_name_list = bloc_name_list()

In [ ]:
bloc_name_list 

['Écarlate et Violet', 'Épée et Bouclier']

# Assignation du nom du bloc aux séries correspondantes

In [ ]:
set_name_list = pokemon_set_df['set_name'].unique()

### Création de listes distincetes en fonction des séries

In [ ]:
SV_set_name_list = set_name_list[:6]
EB_set_name_list = set_name_list[6:]

In [ ]:
SV_set_name_list

array(['Destinées de Paldea', 'Faille Paradoxe', '151',
       'Flammes Obsidiennes', 'Évolutions à Paldea', 'Écarlate et Violet'],
      dtype=object)

In [ ]:
EB_set_name_list

array(['Zénith Suprême', 'Tempête Argentée', 'Origine Perdue',
       'Pokémon GO', 'Astres Radieux', 'Stars Étincelantes',
       'Poing de Fusion', 'Célébrations', 'Évolution Céleste',
       'Styles de Combat', 'Destinées Radieuses', 'Voltage Éclatant',
       'La Voie du Maître', 'Ténèbres Embrasées', 'Clash des Rebelles',
       'Épée et Bouclier'], dtype=object)

### Attribution du bloc correspondant aux séries

In [ ]:
for sv_set_name in SV_set_name_list:
    
    filtered_rows = pokemon_set_df['set_name'] == sv_set_name
    
    pokemon_set_df.loc[filtered_rows, 'bloc_name'] = bloc_name_list[0]

In [ ]:
for eb_set_name in EB_set_name_list:
    
    filtered_rows = pokemon_set_df['set_name'] == eb_set_name
    
    pokemon_set_df.loc[filtered_rows, 'bloc_name'] = bloc_name_list[1]

In [ ]:
pokemon_set_df.to_csv(r"pokemon_set.csv",index=False)

In [ ]:
pokemon_set_df

,set_name,card_title,release_date,cards_count,bloc_name
0,Destinées de Paldea,pomdepik 1 91,,245 cartes (154 secrètes),Écarlate et Violet
1,Destinées de Paldea,foretress ex 2 91,,245 cartes (154 secrètes),Écarlate et Violet
2,Destinées de Paldea,maracachi 3 91,,245 cartes (154 secrètes),Écarlate et Violet
3,Destinées de Paldea,terracool 4 91,,245 cartes (154 secrètes),Écarlate et Violet
4,Destinées de Paldea,terracruel ex 5 91,,245 cartes (154 secrètes),Écarlate et Violet
...,...,...,...,...,...
4610,Épée et Bouclier,zamazenta v 212 202,,216 cartes (14 secrètes),Épée et Bouclier
4611,Épée et Bouclier,ballon 213 202,,216 cartes (14 secrètes),Épée et Bouclier
4612,Épée et Bouclier,ecusson metal 214 202,,216 cartes (14 secrètes),Épée et Bouclier
4613,Épée et Bouclier,canne ordinaire 215 202,,216 cartes (14 secrètes),Épée et Bouclier


# Traitement du nom des cartes 

 Pour obtenir une véracité accrue des données lors du traitement des données, 
 les noms des pokémons doivent être le plus précis possible et aussi suivre les conventions de noms que l'on retrouve sur ebay

## Exemple

Dans pokemon_set_df, si on prend la carte :
* **"lulugabre tg04"**

il serait **plus approprié**, pour **améliorer** la **véracité** des **données sélectionnées**, d'avoir un nom comme:
* **"lulugabre tg04 tg30"** 

En effet ce dernier respecte les **"conventions" d'écriture** des vendeurs ebay français.

## Exemple de conventions d'écriture

* **Carte Pokémon Lugulabre TG04/TG30 Origine Perdue EB11 FR NEUF**

#### Ici on peut voir différents mots clefs
*  **"carte** : en français

* **"lugulabre"** : le nom de pokémon

* **"tg04/tg30"** : le numéro de la carte dans la série

* **"Origine Perdue"** : le nom de la série 

* **"EB11"** : le nom de la série en abrégé

* **"FR"** : mot clef très utile pour identifier les cartes françaises

* **"NEUF"** : mot clefs pour désigner l'état de la carte *non utilisé pour l'instant mais permettrait de classer les cartes scrappées avec plus de précisions.* 

## Code pour formatter le nom des cartes

In [ ]:
# Fonction pour formatter le nom des cartes tg (trainer gallery)
def update_card_title_w_tg(df):
    # Créer un dictionnaire pour stocker le nombre de cartes tg par série
    tg_count = {}
    # Parcourir les lignes du DataFrame
    for index, row in df.iterrows():
        # Vérifier si le nom de la carte contient "tg"
        if "tg" in row['card_title']:
            # Extraire le numéro de la carte tg
            tg_num = int(row['card_title'].split()[-1][2:])
            # Mettre à jour le nombre total de cartes tg dans cette série
            tg_count.setdefault(row['set_name'], 0)
            tg_count[row['set_name']] = max(tg_count[row['set_name']], tg_num)
    # Parcourir à nouveau les lignes pour mettre à jour les noms des cartes
    for index, row in df.iterrows():
        # Vérifier si le nom de la carte contient "tg"
        if "tg" in row['card_title']:
            # Extraire le numéro de la carte tg
            tg_num = int(row['card_title'].split()[-1][2:])
            # Mettre à jour le nom de la carte en ajoutant le rapport
            df.at[index, 'card_title'] += f" tg{tg_count[row['set_name']]}"
    return df

In [ ]:
def update_card_title_w_gg(df):
    # Créer un dictionnaire pour stocker le nombre de cartes gg par série
    gg_count = {}
    
    # Parcourir les lignes du DataFrame
    for index, row in df.iterrows():
        # Vérifier si le nom de la carte contient "gg" et exclure les exceptions
        if "gg" in row['card_title'] and row['card_title'] != "baggiguane 98 203" and row['card_title'] != "baggaid 99 203":
            # Extraire le numéro de la carte gg
            gg_num_str = row['card_title'].split()[-1][2:]
            
            # Vérifier si la chaîne extraite est non vide
            if gg_num_str:
                gg_num = int(gg_num_str)
                
                # Mettre à jour le nombre total de cartes gg dans cette série
                gg_count.setdefault(row['set_name'], 0)
                gg_count[row['set_name']] = max(gg_count[row['set_name']], gg_num)
    
    # Parcourir à nouveau les lignes pour mettre à jour les noms des cartes
    for index, row in df.iterrows():
        # Vérifier si le nom de la carte contient "gg" et exclure les exceptions
        if "gg" in row['card_title'] and row['card_title'] != "baggiguane 98 203" and row['card_title'] != "baggaid 99 203":
            # Extraire le numéro de la carte gg
            gg_num_str = row['card_title'].split()[-1][2:]
            
            # Vérifier si la chaîne extraite est non vide
            if gg_num_str:
                gg_num = int(gg_num_str)
                
                # Mettre à jour le nom de la carte en ajoutant le rapport
                df.at[index, 'card_title'] += f" gg{gg_count[row['set_name']]}"
    
    return df

In [ ]:
def update_card_title_w_sv(df):
    # Créer un dictionnaire pour stocker le nombre de cartes sv par série
    sv_count = {}
    
    # Parcourir les lignes du DataFrame
    for index, row in df.iterrows():
        # Vérifier si le nom de la carte contient "sv"
        if "sv" in row['card_title']:
            # Extraire le numéro de la carte sv
            sv_num = int(row['card_title'].split()[-1][2:])
            # Mettre à jour le nombre total de cartes sv dans cette série
            sv_count.setdefault(row['set_name'], 0)
            sv_count[row['set_name']] = max(sv_count[row['set_name']], sv_num)
    
    # Parcourir à nouveau les lignes pour mettre à jour les noms des cartes
    for index, row in df.iterrows():
        # Vérifier si le nom de la carte contient "sv"
        if "sv" in row['card_title']:
            # Extraire le numéro de la carte sv
            sv_num = int(row['card_title'].split()[-1][2:])
            # Mettre à jour le nom de la carte en ajoutant le rapport
            df.at[index, 'card_title'] += f" sv{sv_count[row['set_name']]}"
    
    return df

In [ ]:
pokemon_set_df = pokemon_set_df.groupby('set_name').apply(update_card_title_w_tg).reset_index(drop=True)

In [ ]:
pokemon_set_df = pokemon_set_df.groupby('set_name').apply(update_card_title_w_gg).reset_index(drop=True)

In [ ]:
pokemon_set_df = pokemon_set_df.groupby('set_name').apply(update_card_title_w_sv).reset_index(drop=True)

#### Problème avec laggron 64 264 gg4 donc on agit directement sur la ligne concernée

In [ ]:
pokemon_set_df['card_title'].replace('laggron 64 264 gg4', 'laggron 64 264', inplace=True)


### Renommage colonne 'card_title' en 'card_name'

In [ ]:
pokemon_set_df.rename(columns={'card_title': 'card_name'}, inplace=True)

# Exportation du dataframe au format .csv

In [ ]:
pokemon_set_df.to_csv("pokemon_set_formatted.csv",index=False)